Crawling Directly Using Course Discussion Boards, but not Group Discussion Boards

In [26]:
import pandas as pd
import math
import random
import xlwt
from xlrd import open_workbook

import requests
import json
import re

def gather (uri, headers):

    group_data_set = []
    
    r = requests.get(uri, headers=headers)

    #print(dir(r))
    #print(r.links)

    raw = r.json()
    #print(raw)

    for group in raw:
        group_data_set.append(group)

        while r.links['current']['url'] != r.links['last']['url']:  
            r = requests.get(r.links['next']['url'], headers=headers)  
            raw = r.json()  
            
            for group in raw:  
                group_data_set.append(group)  
         
    return group_data_set

#print("Enter the course id")
#course_id =input()

#ICT233
course_ids = [22873, 22901, 24224, 24235]

#ICT133 Jan 2020
#course_ids = [22871, 23706, 22899, 24220, 24232, 24239, 24241, 24243, 24245, 24247, 24248]

wb = xlwt.Workbook()
access_token = '3030~eZsvFEUEwW7lSgaSORJGWZTL6ym1RA4dLcy5cuS2hPF7MFD4Jc3FGRigGEOaWR4K'
headers = { 'Authorization': 'Bearer '+ access_token, } 

for course_id in course_ids:

    ws = wb.add_sheet(str(course_id))
    ws.write(0, 0, 'Id')
    ws.write(0, 1, 'course_id')
    ws.write(0, 2, 'Title')
    ws.write(0, 3, 'Message')
    ws.write(0, 4, 'Author')
    ws.write(0, 5, 'Date')
    ws.write(0, 6, 'Time')
    ws.write(0, 7, 'Parent_Id')
    
    row = 1

    uri2 = 'https://canvas.suss.edu.sg/api/v1/courses/' + str(course_id) + '/discussion_topics?per_page=50'
    group_topics = gather(uri2, headers)

    for each_topic in group_topics:
    
        topic_id = each_topic['id']
        
        if topic_id:
            
            author = each_topic['author']
            
            topic_id = each_topic['id']
            title = each_topic['title']
            print(title)
            message=each_topic['message']
            
            posted_datetime = each_topic['posted_at']
            if posted_datetime:
                aList = re.findall(r"[0-9|\-|\:]+", posted_datetime)
            else:
                aList[0] = "2020"
                aList[1] = "Jan"
            #record = str(topic_id) + '\t' + str(group_name) + '\t' + str(title) + '\t' + str(message)[0:15] + '\t' + str(author['display_name']) + '\t' + str(aList[0]) + '\t' + str(aList[1]) + '\t' + 'null' 
            #print(record)       
            
            ws.write(row, 0, topic_id)
            ws.write(row, 1, str(course_id))
            ws.write(row, 2, title)
            ws.write(row, 3, message)
            if (author):
                ws.write(row, 4, author['display_name'])
            else:
                ws.write(row, 4, 'Tacher')
            #s.write(row, 4, 'Tacher')
            ws.write(row, 5, aList[0])
            ws.write(row, 6, aList[1])
            ws.write(row, 7, "null")
           
            row += 1
    
            uri3 = 'https://canvas.suss.edu.sg/api/v1/courses/' + str(course_id) + '/discussion_topics/' + str(topic_id) + '/entries?per_page=50'

            topic_entries = gather(uri3, headers)

            for each_entry in topic_entries:

                entry_id = each_entry['id']
                
                if entry_id:
                
                    author = each_entry['user']
                    entry_id = each_entry['id']
                    entry_message = each_entry['message']
                    entry_posted_datetime = each_entry['created_at']
                    bList = re.findall(r"[0-9|\-|\:]+", entry_posted_datetime)
                    #record = str(entry_id) + '\t' + str(group_name) + '\t' + str(title) + '\t' + str(entry_message)[0:15] + '\t' + str(author['display_name']) + '\t' + str(bList[0]) + '\t' + str(bList[1]) + '\t' + str(topic_id) 
                    #print(record)                   

                    ws.write(row, 0, entry_id)
                    ws.write(row, 1, course_id)
                    ws.write(row, 2, title)
                    ws.write(row, 3, entry_message)
                    ws.write(row, 4, author['display_name'])
                    ws.write(row, 5, bList[0])
                    ws.write(row, 6, bList[1])
                    ws.write(row, 7, topic_id)

                    row += 1
                    
                    uri4 = 'https://canvas.suss.edu.sg/api/v1/courses/' + str(course_id) + '/discussion_topics/' + str(topic_id) + '/entries/' + str(entry_id) + '/replies?per_page=50'

                    replies_to_entry = gather(uri4, headers)
                    
                    for each_reply in replies_to_entry:
                        
                        reply_id = each_reply['id']
                            
                        if reply_id:
                            
                            author = each_reply['user']
                            reply_id = each_reply['id']
                            reply_message = each_reply['message']
                            reply_posted_datetime = each_reply['created_at']
                            cList = bList = re.findall(r"[0-9|\-|\:]+", reply_posted_datetime)
                            #record = str(reply_id) + '\t' + str(group_name) + '\t' + str(title) + '\t' + str(reply_message)[0:15] + '\t' + str(author['display_name']) + '\t' + str(cList[0]) + '\t' + str(cList[1]) + '\t' + str(entry_id) 
                            #print(record)
                            
                            ws.write(row, 0, reply_id)
                            ws.write(row, 1, course_id)
                            ws.write(row, 2, title)
                            ws.write(row, 3, reply_message)
                            ws.write(row, 4, author['display_name'])
                            ws.write(row, 5, cList[0])
                            ws.write(row, 6, cList[1])
                            ws.write(row, 7, entry_id)

                            row += 1

                            #print(json.dumps(each_reply, indent = 4, sort_keys=False))


                            
#wb.save("./ICT133 Jan 2020/ICT133forums.csv")                                
wb.save("./ICT233 Jan 2020/ICT233forums.csv")     

Tan Wei Zhong Benedict - TMA Q3
Tai Ngee Chia - TMA-03
Mohamad Rizuan bin Mazlan - TMA-3
Mindy Toh En Min-TMA-3
Rebecca Low Man Ting TMA Q3b
Tan Meng Cheong TMA Q3
Htin Aung Kyaw - TMA-Q3(b)
Azlina Binte Abdul Rahman - TMA-3
Lian Jing Rui (TMA QN3)
Jovi Tan Yong Jie-TMA-3
Siti Nur Shahirah Binte Abdul Rahman - TMA - 3
Chia Wan Quan - TMA 3
IVAN GOH CHENG YU-TMA-3
Nur Amirah - TMA Question 3b
WAN LI HANG, CHRISTINA - TMA-3
ANDREW CHANDAN DEVASAGAYAM-TMA-3
ONG KOK HWEE-TMA-3
<Tham Shermaine-TMA-3>
KohYaoChiangBrianston-TMA-3
ADRIANLIMZHENGTING-TMA-3
CHINYIBING-TMA-3
HUMSEERA FARVIN - TMA 3B 
Michelle Lim Jia Hui - TMA -3b
Lim Jin Jye -TMA-3
Tai Ye Rong -TMA-3
Loh Han Tiong - TMA -3
Cheong Kok Chue - TMA - 3
Tan Zhi Yee_TMAQn3b
NG CHEE SIAN (HUANG ZHIXIAN) TMA-3 
Quek Yeow Rong TMA 3
 Marcus Goh - TMA Q3: Pros/Cons between SQL database and Pandas Dataframe
TMA Question 3(b)
Getting the average number of GET and POST request on a monthly base
TMA Q2(e)(i-iii)
TMA: Q3d Part (i) Calculate th

In [23]:
import os
os.getcwd()

uri10 = 'https://canvas.suss.edu.sg/api/v1/courses/22871/discussion_topics?per_page=50'

group_topics = gather(uri10, headers)

for topics in group_topics:
    print(topics['id'])
    print(topics['posted_at'])


uri3 = 'https://canvas.suss.edu.sg/api/v1/courses/' + str(course_id) + '/discussion_topics/' + str(topic_id) + '/entries?per_page=50'
    
uri11 = 'https://canvas.suss.edu.sg/api/v1/courses/22871/discussion_topics/121269/entries?per_page=50'

topic_entries = gather(uri11, headers)

for entry in topic_entries:
    author = entry['user']
    print(author['display_name'])

128927
2020-02-06T04:33:24Z
128431
2020-02-04T00:16:52Z
121270
None
121269
None
KOH JING WEN
LI CHEN
LI CHEN
SAMSUL BIN KASAN
MUHAMMAD ZULHILMI BIN MAZLI
HARBINDER SINGH S/O SHUNTO SINGH
WESLEY TEE JUN QUAN
YIM JUN HOW BRIAN
WESLEY TEE JUN QUAN
TAN ZEKAI, MARCUS
YAP ZU QING
TAN ZEKAI, MARCUS
NG WEI LIANG
LIM YEW FAI DEREK (LIN YAOHUI)
LIM ZI YUE LESTER
YIM JUN HOW BRIAN
SANTIAGO LOREINE SANTOS
VIKNESANN MAHALINGAMM
VIKNESANN MAHALINGAMM
VIKNESANN MAHALINGAMM
VIKNESANN MAHALINGAMM
PAUL TAN PEI YANG
YAP ZU QING
CHEN PEI YI
CHEN PEI YI
RIDWAN BIN MOSTAPAH
RIDWAN BIN MOSTAPAH
JEREMY TOH HONG YAO
JEREMY TOH HONG YAO
JEREMY TOH HONG YAO
LOUISA KHO HUI MIN
RAJA SWAMINATHAN
HARBINDER SINGH S/O SHUNTO SINGH
WANG YUEPENG
CHANG PEARLYN
CHANG PEARLYN
KAZUSHI FUJIWARA (TENG YUAN HE ZHI)
HO WILLY
WANG YUEPENG
TAN HUA SOON, WINSTON
NG WEI LIANG
CHEONG YICK HENG, EDMUND
CHEONG YICK HENG, EDMUND
CHAN ZI MIN, EDWIN
GANESH S/O BASKARAN
LOUISA KHO HUI MIN
TAN JUI SENG DARREN
ANG CHIN HENG
MUHAMMAD SYAFIQ 

In [11]:
print (group_topics)

[{'id': 128927, 'title': 'Technical Guide on Submission of Online Test for TMA Question 2', 'last_reply_at': '2020-03-07T04:29:11Z', 'created_at': '2020-02-06T04:16:46Z', 'delayed_post_at': None, 'posted_at': '2020-02-06T04:33:24Z', 'assignment_id': None, 'root_topic_id': None, 'position': None, 'podcast_has_student_posts': False, 'discussion_type': 'threaded', 'lock_at': None, 'allow_rating': True, 'only_graders_can_rate': False, 'sort_by_rating': False, 'is_section_specific': False, 'user_name': 'JUSTIN SEE TIONG BENG (SST)', 'discussion_subentry_count': 14, 'permissions': {'attach': True, 'update': True, 'reply': True, 'delete': True}, 'require_initial_post': None, 'user_can_see_posts': True, 'podcast_url': None, 'read_state': 'read', 'unread_count': 0, 'subscribed': True, 'attachments': [], 'published': True, 'can_unpublish': False, 'locked': False, 'can_lock': True, 'comments_disabled': False, 'author': {'id': 60144, 'display_name': 'JUSTIN SEE TIONG BENG', 'avatar_image_url': 'ht